### read all handbooks into sentences

In [ ]:
from PyPDF2 import PdfReader
import PyPDF2
import re
import os
from regex import F
# import spacy
from tqdm.auto import tqdm
import json
# nlp = spacy.load('en_core_web_sm')

base_dir = "../../data/hdt/handbook/raw/"
books = os.listdir(base_dir)
paras = {}
pbar = tqdm(total = 16316)
change_cols = re.compile(r"\s*\n")
mutil_spaces = re.compile(r"\s+")
sens_interval = re.compile(r"\.|\?|\!")
for book in tqdm(books):
    state, booktype = book.split("-")[0], book.split("-")[1]
    if state in paras:
        paras[state][booktype] = []
    else:
        paras[state] = {}
        paras[state][booktype] = []
    inputpdf = PyPDF2.PdfReader(base_dir + book)
    pages = inputpdf.pages
    for page in pages:
        text = page.extract_text()
        text = re.sub(change_cols, "", text)
        text = re.sub(mutil_spaces, " ", text)
        sens = [i + "." for i in re.split(sens_interval, text) if len(i) > 20]

        paras[state][booktype].extend(sens)
        pbar.update()

with open("../../data/hdt/handbook/ori_sens.json", "w")as f:
    json.dump(paras, f, ensure_ascii=False)

### <font color=yellow>the distribution of sentence numbers (before filtering)</font>

In [2]:
sens_num = 0
for state, a in paras.items():
    for typee, b in a.items():
        print(state + "-" + typee + "-" + str(len(b)))
        sens_num += len(b)
print(sens_num)

ak-motorcycle-940
940


### Filter the sentences by grammar checking

In [ ]:
from happytransformer import HappyTextToText, TTSettings

happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")

args = TTSettings(num_beams=5, min_length=1)

result = happy_tt.generate_text("grammar: ClothingCloth ing can help protect you in a cras h.", args=args)

def valid(sentence):
    good_num = 0
    total_num = 0
    for i in sentence:
        if i.isalpha() or i.isdigit() or i == " ":
            good_num += 1
        total_num += 1
    
    return (good_num/total_num) > 0.9 and len(sentence) > 20
pbar = tqdm(total=27483)

dataset = {}
for state, a in tqdm(paras.items()):
    for typee, sentences in tqdm(a.items()):
        if state in dataset:
            dataset[state][typee] = []
        else:
            dataset[state] = {}
            dataset[state][typee] = []
        for sen in sentences:
            sen = str(sen)
            if valid(sen):
                sen = happy_tt.generate_text("grammar: " + sen, args=args)
                dataset[state][typee].append(sen)
                pbar.update()

### <font color=yellow>the distribution of sentence numbers (after filtering)</font>

In [4]:
sens_num = 0
for state, a in dataset.items():
    for typee, b in a.items():
        print(state + "-" + typee + "-" + str(len(b)))
        sens_num += len(b)

ak-motorcycle-834


### save the paragraphs (after filtering the sentences)

In [ ]:
import json
files = [str(5*i) for i in range(10)]
all_paras = {}
for f in files:
    with open("../../data/hdt/handbook/" + f + ".json")as f:
        data = json.load(f)
    for k1, v1 in data.items():
        all_paras[k1] = {}
        for k, v in v1.items():
            all_paras[k1][k] = []
            sens_num = len(v)
            para_num = sens_num//10 + 1
            for i in range(para_num):
                if i != para_num - 1:
                    para = " ".join(v[10*i:10*i+10])
                else:
                    para = " ".join(v[10*i:-1])
                all_paras[k1][k].append(para)
with open("../../data/hdt/handbook/paras/paras.json", "w")as f:
    json.dump(all_paras, f)